# Readme 

Ce notebook est pour faire une modeèle de machine learning pour prédire si une rdv sera absent ou non.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df2 = pd.read_csv('../donnees_nettroye/2023_gold.csv')

In [7]:
df3 = pd.read_csv('../données_brut/export_rdv_2023-01-01-2023-12-31.csv', sep=';')
df3['Date de saisie'] = df3['Date de saisie'].str[:10]
new_df = pd.DataFrame({ 'Id': df3['Id'], 'Date de saisie': df3['Date de saisie'], 'Date de début': df3['Date de début']})

new_df['Date de saisie'] = pd.to_datetime(new_df['Date de saisie'])
new_df['Date de début'] = pd.to_datetime(new_df['Date de début'], format = "%d/%m/%Y")
new_df['Difference (days)'] = (new_df['Date de début'] - new_df['Date de saisie']).dt.days
print(new_df.head())

           Id Date de saisie Date de début  Difference (days)
0  4276042583     2022-12-29    2023-01-02                  4
1  4279470930     2023-01-01    2023-01-02                  1
2  4192809235     2022-11-30    2023-01-02                 33
3  4279779300     2023-01-01    2023-01-02                  1
4  4277334926     2022-12-30    2023-01-02                  3


/tmp/ipykernel_72801/2580974649.py:1: DtypeWarning: Columns (45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('../données_brut/export_rdv_2023-01-01-2023-12-31.csv', sep=';')


In [8]:
df = pd.merge(df2, new_df[['Id','Difference (days)']], on='Id')

### Sub échantillonage

In [10]:
absence_non_excuse = df[df['Statut'] == 'Absent non excusé']
nb_absence_non_excuse = len(absence_non_excuse)
present = df[df['Statut'] == 'Présent']
present_sample = present.sample(n=nb_absence_non_excuse, random_state=42)
absence_exc = df[df['Statut'] == 'Absent excusé']
resultat = pd.concat([absence_non_excuse, present_sample, absence_exc])
print(resultat['Statut'].value_counts())

Statut
Absent non excusé    1880
Présent              1880
Absent excusé        1441
Name: count, dtype: int64


In [15]:
resultat.head()

,Id,Date de début,Début,Durée du RDV,Motif du RDV,RDV Internet,Nouveau patient,Statut,Civilité,Adresse complète,Age,Difference (days)
10,4206108143,workday,matin,20,Première consultation d'allergologie,Oui,Oui,Absent non excusé,Mme,"27 av Ledru-Rollin , 75012 PARIS",50.0,28
11,4279406927,workday,matin,15,Injection,Oui,Non,Absent non excusé,Mme,"153 Rue de Bagnolet , 75020 PARIS",21.0,1
44,4275454252,workday,matin,20,Première consultation de gynécologie,Oui,Non,Absent non excusé,M.,"4 rue Victor Hugo la queue-en-Brie , 94510 LA ...",22.0,4
46,4280583031,workday,matin,15,Consultation de suivi d'allergologie,Oui,Non,Absent non excusé,M.,"18bis Avenue du Château, 94300 Vincennes",30.0,0
49,4161118766,workday,matin,15,Consultation de suivi d'allergologie,Oui,Non,Absent non excusé,Mme,"Av général Leclerc , 75014 PARIS",28.0,43


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = resultat.drop(['Statut'], axis=1).drop(['Id'], axis=1).drop(['Adresse complète'], axis=1).drop(['Durée du RDV'], axis=1)
X = pd.get_dummies(X)

y = resultat['Statut']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
X_train.head()

,Age,Difference (days),Date de début_weekend,Date de début_workday,Début_apres-midi,Début_matin,Début_soir,Motif du RDV_1re injection (avec 2de injection) vaccin COVID-19 (Novavax),Motif du RDV_2de injection vaccin COVID-19 (Moderna),Motif du RDV_2de injection vaccin COVID-19 (Pfizer-BioNTech),...,Motif du RDV_consultation (cabine),Motif du RDV_enfant/ado - consultation de médecine générale,Motif du RDV_téléconsultation augmentée,Motif du RDV_Électrocardiogramme (ECG),RDV Internet_Non,RDV Internet_Oui,Nouveau patient_Non,Nouveau patient_Oui,Civilité_M.,Civilité_Mme
19502,25.0,21,False,True,True,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
1467,28.0,1,False,True,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,True,False
7421,43.0,13,False,True,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,True
2608,37.0,1,False,True,True,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,True
18947,10.0,0,False,True,True,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,True


### Random Forest

In [31]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


# Modèle avec pondération des classes
model = RandomForestClassifier(class_weight={'Absent excusé': 1, 'Absent non excusé': 0.5, 'Présent': 1})

# Utilisation de GridSearchCV pour optimiser les hyperparamètres
param_grid = {
    'n_estimators': [10,30,50,100,200],
    'max_depth': [5,10, 20],
}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)
print(grid_search.best_params_)

# Prédire et évaluer
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_train)
print(classification_report(y_train, y_pred))

{'max_depth': 10, 'n_estimators': 30}
                   precision    recall  f1-score   support

    Absent excusé       0.68      0.46      0.55      1140
Absent non excusé       0.98      0.10      0.18      1481
          Présent       0.45      0.96      0.62      1539

         accuracy                           0.51      4160
        macro avg       0.70      0.50      0.45      4160
     weighted avg       0.70      0.51      0.44      4160



In [32]:
best_model = RandomForestClassifier(class_weight={'Absent excusé': 1, 'Absent non excusé': 0.5, 'Présent': 1})
best_model.set_params(**grid_search.best_params_)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

                   precision    recall  f1-score   support

    Absent excusé       0.57      0.31      0.40       301
Absent non excusé       0.53      0.04      0.08       399
          Présent       0.36      0.91      0.52       341

         accuracy                           0.40      1041
        macro avg       0.49      0.42      0.33      1041
     weighted avg       0.49      0.40      0.32      1041



### regression logistique

In [34]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight={'Absent excusé': 1, 'Absent non excusé': 0.5, 'Présent': 1})

# Utilisation de GridSearchCV pour optimiser les hyperparamètres
param_grid = {
    'C': [0.1, 1, 10, 100],
}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)
print(grid_search.best_params_)
# Prédire et évaluer
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X)
print(classification_report(y, y_pred))





/home/wang/miniforge3/envs/optim/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/wang/miniforge3/envs/optim/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

{'C': 0.1}
                   precision    recall  f1-score   support

    Absent excusé       0.45      0.43      0.44      1441
Absent non excusé       0.52      0.01      0.01      1880
          Présent       0.42      0.85      0.56      1880

         accuracy                           0.43      5201
        macro avg       0.46      0.43      0.34      5201
     weighted avg       0.47      0.43      0.33      5201

